# Fine-tune Gemma models in Keras using LoRA on your laptop


## Overview

this tutorial can run on WSL of your laptop. you should install WSL、CUDA、VS Code and PyTorch

## Setup

### Configure your kaggle API key and wandb key

To use Gemma, you must provide your Kaggle username and a Kaggle API key. To use wandb to monitor, you must provide wandb API key.

you can save the key information in .env


### Set environment variables

Set environment variables for `wandb`, `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [ ]:
!pip install -q -U python-dotenv

In [ ]:
import os
from dotenv import find_dotenv,load_dotenv
import wandb

# Note: set in .env
# vars as appropriate for your system.

env=load_dotenv(find_dotenv())


os.environ["KAGGLE_USERNAME"] 
os.environ["KAGGLE_KEY"] 
wb_token=os.environ["wandb"]
# wandb.login(key=wb_token)


### Install dependencies

Install Keras, KerasNLP

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [ ]:
import keras
import keras_nlp
print(keras.__version__)
print(keras_nlp.__version__)

### Set a backend


For this tutorial, configure the backend for PyTorch.

In [ ]:
os.environ["KERAS_BACKEND"] = "torch"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [ ]:
import keras
import keras_nlp

## Load Dataset

In [ ]:
!pip install -q -U datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("databricks/databricks-dolly-15k")
#dataset['train']['instruction'][1]
#dataset['train']['response'][1]
#dataset['train']['context']

Preprocess the data. This tutorial uses a subset of 5000 training examples to execute the notebook. Consider using more training data for higher quality fine-tuning.

In [ ]:
data = []
for d in dataset['train']:
    # Filter out examples with context, to keep it simple.
    if d["context"]:
        continue
    # Format the entire example as a single string.
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(template.format(**d))

# use 5000 training examples, to keep it fast.
data = data[:5000]
#data

## Load Model

Use KerasNLP to load Gemma model.

Create the model using the `from_preset` method:

In [ ]:

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

The `from_preset` method instantiates the model from a preset architecture and weights. In the code above, the string "gemma_2b_en" specifies the preset architecture — a Gemma model with 2 billion parameters.

NOTE: A Gemma model with 7
billion parameters is also available. To run the larger model in Colab, you need access to the premium GPUs available in paid plans. Alternatively, you can perform [distributed tuning on a Gemma 7B model](https://ai.google.dev/gemma/docs/distributed_tuning) on Kaggle or Google Cloud.

## Inference before fine tuning

In this section, you will query the model with various prompts to see how it responds.


### Europe Trip Prompt

Query the model for suggestions on what to do on a trip to Europe.

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

The model just responds with a recommendation to take a trip to Europe.

### ELI5 Photosynthesis Prompt

Prompt the model to explain photosynthesis in terms simple enough for a 5 year old child to understand.


In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

The responses contains words that might not be easy to understand for a child such as chlorophyll, glucose, etc.

## LoRA Fine-tuning

To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA) using the Databricks Dolly 15k dataset.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.5 billion to 1.3 million).

In [ ]:
from wandb.keras import WandbMetricsLogger,WandbCallback
import wandb

wandb.login(key=os.environ["wandb"])

wandb.init(config={"bs":12})


# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=2, batch_size=2, callbacks=[WandbMetricsLogger(log_freq=4)])

## Inference after fine-tuning
After fine-tuning, responses follow the instruction provided in the prompt.

### Europe Trip Prompt


In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

The model now recommends places to visit in Europe.

### ELI5 Photosynthesis Prompt


In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

The model now explains photosynthesis in simpler terms.

Note that for demonstration purposes, this tutorial fine-tunes the model on a small subset of the dataset for just one epoch and with a low LoRA rank value. To get better responses from the fine-tuned model, you can experiment with:

1. Increasing the size of the fine-tuning dataset
2. Training for more steps (epochs)
3. Setting a higher LoRA rank
4. Modifying the hyperparameter values such as `learning_rate` and `weight_decay`.

reference code：https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb
